## <font color = blue> Streaming public tweets related to Ukraine War from Twitter using tweepy
### Created by Nisha Mundanthara Suresh Babu

In [2]:
import pandas as pd
import numpy as np
import json
import nltk #for stop words
nltk.download('stopwords')
import string
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import geocoder#geting country name
import pycountry #country names
import re #regular expressions
import tweepy
import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Nisha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#create the key and token variables and assign them values
consumer_key = "NEL2JGetq42m8mZ2QhjWjCK36"
consumer_secret = "47FncwcJjKlsuWDwDA3P3AAqLcbgaG9lPLQ9WURLKczGYq5j3z"
access_token = "1465980889951842307-CTP5cHkGNHBU8dblcQorqqoPjlzg6S"
access_token_secret = "i7WrqTOMm0PmJE0MYgOPeqgWwrE5qoCHjkjO4qphckbuL"

In [ ]:
#now set authentication
#new method to authenticate
auth = tweepy.OAuth1UserHandler(consumer_key, consumer_secret)

### <font color = blue> The tweets were streamed for 12 hours.

### <font color = blue>  Start time: Sun May 01 16:33
###  <font color = blue> End time:Mon May 02 04:33

In [ ]:
class myStreamListener(tweepy.Stream):
        
        def on_connect(self):
            # Called initially to connect to the Streaming API
            print("You are now connected to the streaming API.")
            print("------------------")
            print("Start Time: ", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        
        def __init__(self, consumer_key, consumer_secret, access_token, access_token_secret):
            super(myStreamListener, self).__init__(consumer_key, consumer_secret, access_token, access_token_secret)
            #self.num_tweets = 0
            self.info = []
            self.start_time = time.time() #to get start time
            self.last = 0
            self.end_time=43200 #12 hours
        
        def on_request_error(status_code):
        #def on_error(self, status_code):
            # On error - if an error occurs, display the error / status code
            print('An Error has occured: ' + repr(status_code))
            return False
    

        def on_data(self, data):
            
            
            try:
                timer = time.time() - self.start_time
                if timer <= self.end_time: 
                
                    # Decode the JSON from Twitter
                    datajson = json.loads(data)
                   # append tweet information to the list info
                    self.info.append([datajson['created_at'],datajson['text'],datajson['user']['location'],datajson['user']['name'],datajson['place']])
                

                    #grab the 'created_at' data from the Tweet to use for display
                    self.last = datetime.now().strftime('%Y-%m-%d %H:%M:%S') #to get the end time
                
                else:
                    
                    self.disconnect() #disconnect when 12 hours streamed
                      
                    # export df to a CSV file, leaving out the row "labels" (row numbers)
                    df = pd.DataFrame(self.info, columns = ['create_at','text','location','user name','place'])
                    df.to_csv("tweet_data.csv", ",", index=False)
                    print("End Time: ", self.last)
                    return False
                
            except Exception as e:
                   print(e)
            

In [ ]:
#the init method as per new versio needs the keys and tokens
streamer = myStreamListener(consumer_key='NEL2JGetq42m8mZ2QhjWjCK36',
                            consumer_secret='47FncwcJjKlsuWDwDA3P3AAqLcbgaG9lPLQ9WURLKczGYq5j3z',
                            access_token ='1465980889951842307-CTP5cHkGNHBU8dblcQorqqoPjlzg6S',
                            access_token_secret='i7WrqTOMm0PmJE0MYgOPeqgWwrE5qoCHjkjO4qphckbuL')

#you can add this line if you are filtering stream for a location
# print("Tracking location: " + str('Wales'))

print("Tracking tweets related to Ukraine War")

#to filter stream with a list of keywords
streamer.filter(track = ['#RussianUkraineWar','#UkraineWar','#IStandWithUkraine','#StandWithUkraineFlag','#IStandWithPutin','#IStandWithRussia','#Russia','#Ukraine'],languages=['en'])



## <font color = blue> Preprocess the 'tweet_data.csv'

In [2]:
tweet_df = pd.read_csv('tweet_data.csv') #Read csv
tweet_df.head()

,create_at,text,location,user name,place
0,Sun May 01 16:33:27 +0000 2022,RT @RikeFranke: This is a very thoughtful text.\n#Ukraine #offenerbrief \n\nhttps://t.co/o8OuyJ7...,NaN,Thorsten,NaN
1,Sun May 01 16:33:27 +0000 2022,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,"Vancouver, BC",Citizen of EU,NaN
2,Sun May 01 16:33:27 +0000 2022,"RT @timand2037: ""War-monger! Liar!"" - in #Arensburg #Germany, residents disrupt the rally of For...",NaN,Wasu limsodsai,NaN
3,Sun May 01 16:33:27 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,NaN,karen king,NaN
4,Sun May 01 16:33:27 +0000 2022,RT @ThatKHersch: There have been numerous reports today of one or more ammunition depot #fires i...,NaN,Meneer123,NaN


In [3]:
print('Dataset size:',tweet_df.shape)
print('Columns are:',tweet_df.columns)

Dataset size: (97339, 5)
Columns are: Index(['create_at', 'text', 'location', 'user name', 'place'], dtype='object')


In [4]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97339 entries, 0 to 97338
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   create_at  97339 non-null  object
 1   text       97339 non-null  object
 2   location   54156 non-null  object
 3   user name  97338 non-null  object
 4   place      336 non-null    object
dtypes: object(5)
memory usage: 3.7+ MB


In [5]:
tweet_df.rename(columns={'user name':'user_name'}, inplace=True)

In [6]:
tweet_df = tweet_df.dropna(subset = ['location'])
tweet_df['location']

1                 Vancouver, BC
5                         Italy
6                        London
10                   💙freeworld
12                      Ukraine
                  ...          
97334                  Warszawa
97335                   Calgary
97336                 Ohio, USA
97337               Addis Abeba
97338    Žatec, Česká republika
Name: location, Length: 54156, dtype: object

In [7]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54156 entries, 1 to 97338
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   create_at  54156 non-null  object
 1   text       54156 non-null  object
 2   location   54156 non-null  object
 3   user_name  54156 non-null  object
 4   place      286 non-null    object
dtypes: object(5)
memory usage: 2.5+ MB


In [8]:
# Remove URLs

def remove_url(text):
    text = re.sub(r"http\S+", "", text)
    return text

tweet_df['no_url'] = tweet_df['text'].apply(lambda x: remove_url(x))

In [9]:
#Create a column with number of user mentions in each tweet

#user mentions
#Max character in a twitter username is 15
def mentions(text):
    text = re.findall("@([a-zA-Z0-9]{1,15})", text)
    return text
#To avoid taking @ from email addres use url removed text
tweet_df['Mentions'] = tweet_df['no_url'].apply(lambda x: mentions(x))

def No_User_Mentions(text):
    length = len(set(text))
    return length
    
# Count number of user mentions in each tweet
tweet_df['No_User_Mentions'] = tweet_df['Mentions'].apply(lambda x: No_User_Mentions(x))


tweet_df['No_User_Mentions']

1        1
5        1
6        1
10       1
12       2
        ..
97334    1
97335    1
97336    1
97337    1
97338    1
Name: No_User_Mentions, Length: 54156, dtype: int64

In [10]:
#REmove hashtags and mentions

In [13]:
def remove_hashandmentions(text):
    text = re.sub("(#|@)[A-Za-z0-9_]+","", text)
    return text

tweet_df['cleaned'] = tweet_df['no_url'].apply(lambda x: remove_hashandmentions(x))
tweet_df.head(10)

,create_at,text,location,user_name,place,no_url,Mentions,No_User_Mentions,cleaned
1,Sun May 01 16:33:27 +0000 2022,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,"Vancouver, BC",Citizen of EU,NaN,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,[Striemond],1,"RT : For now 80 people were evacuated from the territory of the plant in , who were held by N..."
5,Sun May 01 16:33:27 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,Italy,umberto,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT : This is how an 11 year old boy sees the world after having been raped by soldier in front ...
6,Sun May 01 16:33:28 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,London,David Granger,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT : This is how an 11 year old boy sees the world after having been raped by soldier in front ...
10,Sun May 01 16:33:30 +0000 2022,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",💙freeworld,BR,NaN,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",[nexta],1,"RT : In occupied territories, military is destroying the medical infrastructure and removing th..."
12,Sun May 01 16:33:32 +0000 2022,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,Ukraine,Ukraine War Art,NaN,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,"[UkraineWarArt, KremlinRussia]",2,RT : Another victim of Putin’s Failed Presidency and illegal war in … Rest in Peace An…
16,Sun May 01 16:33:34 +0000 2022,Fairytale castle 🏰\n🗺📍\n#trakai #lithuania #lietuva\n📸\n#trakaicastle\n♯\n#travel #adventure #vi...,Worldwide,99countries,"{'id': '01cce83d8638e5f7', 'url': 'https://api.twitter.com/1.1/geo/id/01cce83d8638e5f7.json', 'p...",Fairytale castle 🏰\n🗺📍\n#trakai #lithuania #lietuva\n📸\n#trakaicastle\n♯\n#travel #adventure #vi...,[],0,Fairytale castle 🏰\n🗺📍\n \n📸\n\n♯\n …
20,Sun May 01 16:33:37 +0000 2022,RT @CanadianUkrain1: #Japanese volunteers in #Ukraine.\n\nGlory to heroes! ✊🏻🇺🇦 https://t.co/r5Y...,Ukraine,Koenig Segg,NaN,RT @CanadianUkrain1: #Japanese volunteers in #Ukraine.\n\nGlory to heroes! ✊🏻🇺🇦,[CanadianUkrain1],1,RT : volunteers in .\n\nGlory to heroes! ✊🏻🇺🇦
22,Sun May 01 16:33:38 +0000 2022,RT @CanadianUkrain1: This was a fun night of blocking troll accounts and wannabe OSINT analysts ...,Fredericton,Putin is a war criminal. 🇺🇦,NaN,RT @CanadianUkrain1: This was a fun night of blocking troll accounts and wannabe OSINT analysts ...,[CanadianUkrain1],1,RT : This was a fun night of blocking troll accounts and wannabe OSINT analysts that seem to und...
23,Sun May 01 16:33:38 +0000 2022,"RT @MrKovalenko: [2] Yesterday, at the moment of the #Ukraine attack more than 40 high-ranked Ru...",PACA,M.M.MENDOZA,NaN,"RT @MrKovalenko: [2] Yesterday, at the moment of the #Ukraine attack more than 40 high-ranked Ru...",[MrKovalenko],1,"RT : [2] Yesterday, at the moment of the attack more than 40 high-ranked Russian military offic..."
24,Sun May 01 16:33:38 +0000 2022,#Westone supports our brothers in #Ukraine from this unjust war. \n\n#SupportUkraine #Ukraine️war,"Seabrook, New Hampshire",Westone Guitars,NaN,#Westone supports our brothers in #Ukraine from this unjust war. \n\n#SupportUkraine #Ukraine️war,[],0,supports our brothers in from this unjust war. \n\n ️war


In [14]:
#Remove punctations and special symbols from tweet
def remove_punct(text):
    #text = re.sub(r'[^\w\s]', '', text) # remove only punctuations
    text = re.sub(r'[^A-Za-z]+', ' ',text) #remove all special characters including emojis
    return text

tweet_df['cleaned'] = tweet_df['cleaned'].apply(lambda x: remove_punct(x))
tweet_df.head(10)

,create_at,text,location,user_name,place,no_url,Mentions,No_User_Mentions,cleaned
1,Sun May 01 16:33:27 +0000 2022,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,"Vancouver, BC",Citizen of EU,NaN,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,[Striemond],1,RT For now people were evacuated from the territory of the plant in who were held by Naziks
5,Sun May 01 16:33:27 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,Italy,umberto,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT This is how an year old boy sees the world after having been raped by soldier in front of his...
6,Sun May 01 16:33:28 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,London,David Granger,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT This is how an year old boy sees the world after having been raped by soldier in front of his...
10,Sun May 01 16:33:30 +0000 2022,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",💙freeworld,BR,NaN,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",[nexta],1,RT In occupied territories military is destroying the medical infrastructure and removing the eq...
12,Sun May 01 16:33:32 +0000 2022,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,Ukraine,Ukraine War Art,NaN,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,"[UkraineWarArt, KremlinRussia]",2,RT Another victim of Putin s Failed Presidency and illegal war in Rest in Peace An
16,Sun May 01 16:33:34 +0000 2022,Fairytale castle 🏰\n🗺📍\n#trakai #lithuania #lietuva\n📸\n#trakaicastle\n♯\n#travel #adventure #vi...,Worldwide,99countries,"{'id': '01cce83d8638e5f7', 'url': 'https://api.twitter.com/1.1/geo/id/01cce83d8638e5f7.json', 'p...",Fairytale castle 🏰\n🗺📍\n#trakai #lithuania #lietuva\n📸\n#trakaicastle\n♯\n#travel #adventure #vi...,[],0,Fairytale castle
20,Sun May 01 16:33:37 +0000 2022,RT @CanadianUkrain1: #Japanese volunteers in #Ukraine.\n\nGlory to heroes! ✊🏻🇺🇦 https://t.co/r5Y...,Ukraine,Koenig Segg,NaN,RT @CanadianUkrain1: #Japanese volunteers in #Ukraine.\n\nGlory to heroes! ✊🏻🇺🇦,[CanadianUkrain1],1,RT volunteers in Glory to heroes
22,Sun May 01 16:33:38 +0000 2022,RT @CanadianUkrain1: This was a fun night of blocking troll accounts and wannabe OSINT analysts ...,Fredericton,Putin is a war criminal. 🇺🇦,NaN,RT @CanadianUkrain1: This was a fun night of blocking troll accounts and wannabe OSINT analysts ...,[CanadianUkrain1],1,RT This was a fun night of blocking troll accounts and wannabe OSINT analysts that seem to under...
23,Sun May 01 16:33:38 +0000 2022,"RT @MrKovalenko: [2] Yesterday, at the moment of the #Ukraine attack more than 40 high-ranked Ru...",PACA,M.M.MENDOZA,NaN,"RT @MrKovalenko: [2] Yesterday, at the moment of the #Ukraine attack more than 40 high-ranked Ru...",[MrKovalenko],1,RT Yesterday at the moment of the attack more than high ranked Russian military officers gathere...
24,Sun May 01 16:33:38 +0000 2022,#Westone supports our brothers in #Ukraine from this unjust war. \n\n#SupportUkraine #Ukraine️war,"Seabrook, New Hampshire",Westone Guitars,NaN,#Westone supports our brothers in #Ukraine from this unjust war. \n\n#SupportUkraine #Ukraine️war,[],0,supports our brothers in from this unjust war war


In [15]:
#Tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text

tweet_df['Tweet_tokenized'] = tweet_df['cleaned'].apply(lambda x: tokenization(x.lower()))

stopword = nltk.corpus.stopwords.words('english')
#Extend stop words to add rt(indicator for a retweet)
stopword.extend(['rt'])
#Remove stopwords
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
tweet_df['final'] = tweet_df['Tweet_tokenized'].apply(lambda x: remove_stopwords(x))
tweet_df.head(10)

,create_at,text,location,user_name,place,no_url,Mentions,No_User_Mentions,cleaned,Tweet_tokenized,final
1,Sun May 01 16:33:27 +0000 2022,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,"Vancouver, BC",Citizen of EU,NaN,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,[Striemond],1,RT For now people were evacuated from the territory of the plant in who were held by Naziks,"[rt, for, now, people, were, evacuated, from, the, territory, of, the, plant, in, who, were, hel...","[people, evacuated, territory, plant, held, naziks, ]"
5,Sun May 01 16:33:27 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,Italy,umberto,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT This is how an year old boy sees the world after having been raped by soldier in front of his...,"[rt, this, is, how, an, year, old, boy, sees, the, world, after, having, been, raped, by, soldie...","[year, old, boy, sees, world, raped, soldier, front, mother, find, ]"
6,Sun May 01 16:33:28 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,London,David Granger,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT This is how an year old boy sees the world after having been raped by soldier in front of his...,"[rt, this, is, how, an, year, old, boy, sees, the, world, after, having, been, raped, by, soldie...","[year, old, boy, sees, world, raped, soldier, front, mother, find, ]"
10,Sun May 01 16:33:30 +0000 2022,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",💙freeworld,BR,NaN,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",[nexta],1,RT In occupied territories military is destroying the medical infrastructure and removing the eq...,"[rt, in, occupied, territories, military, is, destroying, the, medical, infrastructure, and, rem...","[occupied, territories, military, destroying, medical, infrastructure, removing, equipment, equi..."
12,Sun May 01 16:33:32 +0000 2022,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,Ukraine,Ukraine War Art,NaN,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,"[UkraineWarArt, KremlinRussia]",2,RT Another victim of Putin s Failed Presidency and illegal war in Rest in Peace An,"[rt, another, victim, of, putin, s, failed, presidency, and, illegal, war, in, rest, in, peace, ...","[another, victim, putin, failed, presidency, illegal, war, rest, peace, ]"
16,Sun May 01 16:33:34 +0000 2022,Fairytale castle 🏰\n🗺📍\n#trakai #lithuania #lietuva\n📸\n#trakaicastle\n♯\n#travel #adventure #vi...,Worldwide,99countries,"{'id': '01cce83d8638e5f7', 'url': 'https://api.twitter.com/1.1/geo/id/01cce83d8638e5f7.json', 'p...",Fairytale castle 🏰\n🗺📍\n#trakai #lithuania #lietuva\n📸\n#trakaicastle\n♯\n#travel #adventure #vi...,[],0,Fairytale castle,"[fairytale, castle, ]","[fairytale, castle, ]"
20,Sun May 01 16:33:37 +0000 2022,RT @CanadianUkrain1: #Japanese volunteers in #Ukraine.\n\nGlory to heroes! ✊🏻🇺🇦 https://t.co/r5Y...,Ukraine,Koenig Segg,NaN,RT @CanadianUkrain1: #Japanese volunteers in #Ukraine.\n\nGlory to heroes! ✊🏻🇺🇦,[CanadianUkrain1],1,RT volunteers in Glory to heroes,"[rt, volunteers, in, glory, to, heroes, ]","[volunteers, glory, heroes, ]"
22,Sun May 01 16:33:38 +0000 2022,RT @CanadianUkrain1: This was a fun night of blocking troll accounts and wannabe OSINT analysts ...,Fredericton,Putin is a war criminal. 🇺🇦,NaN,RT @CanadianUkrain1: This was a fun night of blocking troll accounts and wannabe OSINT analysts ...,[CanadianUkrain1],1,RT This was a fun night of blocking troll accounts 

In [16]:

#Finding hashtag frequencies

#tokenise the words to get all hashtags/words (this function makes all words small letters)
tweet_df['tokens'] = tweet_df['text'].apply(lambda x: tokenization(x.lower()))

def freq(text,word):
    count = 0
    for ele in text:
            if ele == word:
                count += 1
    return count

tweet_df['Ukraine'] = tweet_df['tokens'].apply(lambda x: freq(x,'ukraine'))
tweet_df['Russia'] = tweet_df['tokens'].apply(lambda x: freq(x,'russia'))
tweet_df['UkraineWar'] = tweet_df['tokens'].apply(lambda x: freq(x,'ukrainewar'))
tweet_df['IStandWithUkraine'] = tweet_df['tokens'].apply(lambda x: freq(x,'istandwithukraine'))
tweet_df['StandWithUkraineFlag'] = tweet_df['tokens'].apply(lambda x: freq(x,'standwithukraineflag'))
tweet_df['IStandWithPutin'] = tweet_df['tokens'].apply(lambda x: freq(x,'istandwithputin'))
tweet_df['IStandWithRussia'] = tweet_df['tokens'].apply(lambda x: freq(x,'istandwithrussia'))
tweet_df['RussianUkraineWar'] = tweet_df['tokens'].apply(lambda x: freq(x,'russianukrainewar'))
tweet_df.head()



,create_at,text,location,user_name,place,no_url,Mentions,No_User_Mentions,cleaned,Tweet_tokenized,final,tokens,Ukraine,Russia,UkraineWar,IStandWithUkraine,StandWithUkraineFlag,IStandWithPutin,IStandWithRussia,RussianUkraineWar
1,Sun May 01 16:33:27 +0000 2022,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,"Vancouver, BC",Citizen of EU,NaN,RT @Striemond: For now 80 people were evacuated from the territory of the #Azovstal plant in #Ma...,[Striemond],1,RT For now people were evacuated from the territory of the plant in who were held by Naziks,"[rt, for, now, people, were, evacuated, from, the, territory, of, the, plant, in, who, were, hel...","[people, evacuated, territory, plant, held, naziks, ]","[rt, striemond, for, now, 80, people, were, evacuated, from, the, territory, of, the, azovstal, ...",0,0,0,0,0,0,0,0
5,Sun May 01 16:33:27 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,Italy,umberto,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT This is how an year old boy sees the world after having been raped by soldier in front of his...,"[rt, this, is, how, an, year, old, boy, sees, the, world, after, having, been, raped, by, soldie...","[year, old, boy, sees, world, raped, soldier, front, mother, find, ]","[rt, lesiavasylenko, this, is, how, an, 11, year, old, boy, sees, the, world, after, having, bee...",0,1,0,0,0,0,0,0
6,Sun May 01 16:33:28 +0000 2022,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,London,David Granger,NaN,RT @lesiavasylenko: This is how an 11 year old boy sees the world after having been raped by #Ru...,[lesiavasylenko],1,RT This is how an year old boy sees the world after having been raped by soldier in front of his...,"[rt, this, is, how, an, year, old, boy, sees, the, world, after, having, been, raped, by, soldie...","[year, old, boy, sees, world, raped, soldier, front, mother, find, ]","[rt, lesiavasylenko, this, is, how, an, 11, year, old, boy, sees, the, world, after, having, bee...",0,1,0,0,0,0,0,0
10,Sun May 01 16:33:30 +0000 2022,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",💙freeworld,BR,NaN,"RT @nexta_tv: In occupied territories, #Russian military is destroying the medical infrastructur...",[nexta],1,RT In occupied territories military is destroying the medical infrastructure and removing the eq...,"[rt, in, occupied, territories, military, is, destroying, the, medical, infrastructure, and, rem...","[occupied, territories, military, destroying, medical, infrastructure, removing, equipment, equi...","[rt, nexta_tv, in, occupied, territories, russian, military, is, destroying, the, medical, infra...",0,0,0,0,0,0,0,0
12,Sun May 01 16:33:32 +0000 2022,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,Ukraine,Ukraine War Art,NaN,RT @UkraineWarArt: Another victim of @KremlinRussia Putin’s Failed Presidency and illegal war in...,"[UkraineWarArt, KremlinRussia]",2,RT Another victim of Putin s Failed Presidency and illegal war in Rest in Peace An,"[rt, another, victim, of, putin, s, failed, presidency, and, illegal, war, in, rest, in, peace, ...","[another, victim, putin, failed, presidency, illegal, war, rest, peace, ]","[rt, ukrainewarart, another, victim, of, kremlinrussia, putin, s, failed, presidency, and, illeg...",1,0,0,0,0,0,0,0


In [17]:
#ISO Country codes
CC = {
    "AF": "AFGHANISTAN",
    "AX": "ÅLAND ISLANDS",
    "AL": "ALBANIA",
    "DZ": "ALGERIA",
    "AS": "AMERICAN SAMOA",
    "AD": "ANDORRA",
    "AO": "ANGOLA",
    "AI": "ANGUILLA",
    "AQ": "ANTARCTICA",
    "AG": "ANTIGUA AND BARBUDA",
    "AR": "ARGENTINA",
    "AM": "ARMENIA",
    "AW": "ARUBA",
    "AU": "AUSTRALIA",
    "AT": "AUSTRIA",
    "AZ": "AZERBAIJAN",
    "BS": "BAHAMAS",
    "BH": "BAHRAIN",
    "BD": "BANGLADESH",
    "BB": "BARBADOS",
    "BY": "BELARUS",
    "BE": "BELGIUM",
    "BZ": "BELIZE",
    "BJ": "BENIN",
    "BM": "BERMUDA",
    "BT": "BHUTAN",
    "BO": "BOLIVIA, PLURINATIONAL STATE OF",
    "BQ": "BONAIRE, SINT EUSTATIUS AND SABA",
    "BA": "BOSNIA AND HERZEGOVINA",
    "BW": "BOTSWANA",
    "BV": "BOUVET ISLAND",
    "BR": "BRAZIL",
    "IO": "BRITISH INDIAN OCEAN TERRITORY",
    "BN": "BRUNEI DARUSSALAM",
    "BG": "BULGARIA",
    "BF": "BURKINA FASO",
    "BI": "BURUNDI",
    "KH": "CAMBODIA",
    "CM": "CAMEROON",
    "CA": "CANADA",
    "CV": "CAPE VERDE",
    "KY": "CAYMAN ISLANDS",
    "CF": "CENTRAL AFRICAN REPUBLIC",
    "TD": "CHAD",
    "CL": "CHILE",
    "CN": "CHINA",
    "CX": "CHRISTMAS ISLAND",
    "CC": "COCOS (KEELING) ISLANDS",
    "CO": "COLOMBIA",
    "KM": "COMOROS",
    "CG": "CONGO",
    "CD": "CONGO, THE DEMOCRATIC REPUBLIC OF THE",
    "CK": "COOK ISLANDS",
    "CR": "COSTA RICA",
    "CI": "CÔTE D'IVOIRE",
    "HR": "CROATIA",
    "CU": "CUBA",
    "CW": "CURAÇAO",
    "CY": "CYPRUS",
    "CZ": "CZECH REPUBLIC",
    "DK": "DENMARK",
    "DJ": "DJIBOUTI",
    "DM": "DOMINICA",
    "DO": "DOMINICAN REPUBLIC",
    "EC": "ECUADOR",
    "EG": "EGYPT",
    "SV": "EL SALVADOR",
    "GQ": "EQUATORIAL GUINEA",
    "ER": "ERITREA",
    "EE": "ESTONIA",
    "ET": "ETHIOPIA",
    "FK": "FALKLAND ISLANDS (MALVINAS)",
    "FO": "FAROE ISLANDS",
    "FJ": "FIJI",
    "FI": "FINLAND",
    "FR": "FRANCE",
    "GF": "FRENCH GUIANA",
    "PF": "FRENCH POLYNESIA",
    "TF": "FRENCH SOUTHERN TERRITORIES",
    "GA": "GABON",
    "GM": "GAMBIA",
    "GE": "GEORGIA",
    "DE": "GERMANY",
    "GH": "GHANA",
    "GI": "GIBRALTAR",
    "GR": "GREECE",
    "GL": "GREENLAND",
    "GD": "GRENADA",
    "GP": "GUADELOUPE",
    "GU": "GUAM",
    "GT": "GUATEMALA",
    "GG": "GUERNSEY",
    "GN": "GUINEA",
    "GW": "GUINEA-BISSAU",
    "GY": "GUYANA",
    "HT": "HAITI",
    "HM": "HEARD ISLAND AND MCDONALD ISLANDS",
    "VA": "HOLY SEE (VATICAN CITY STATE)",
    "HN": "HONDURAS",
    "HK": "HONG KONG",
    "HU": "HUNGARY",
    "IS": "ICELAND",
    "IN": "INDIA",
    "ID": "INDONESIA",
    "IR": "IRAN, ISLAMIC REPUBLIC OF",
    "IQ": "IRAQ",
    "IE": "IRELAND",
    "IM": "ISLE OF MAN",
    "IL": "ISRAEL",
    "IT": "ITALY",
    "JM": "JAMAICA",
    "JP": "JAPAN",
    "JE": "JERSEY",
    "JO": "JORDAN",
    "KZ": "KAZAKHSTAN",
    "KE": "KENYA",
    "KI": "KIRIBATI",
    "KP": "KOREA, DEMOCRATIC PEOPLE'S REPUBLIC OF",
    "KR": "KOREA, REPUBLIC OF",
    "KW": "KUWAIT",
    "KG": "KYRGYZSTAN",
    "LA": "LAO PEOPLE'S DEMOCRATIC REPUBLIC",
    "LV": "LATVIA",
    "LB": "LEBANON",
    "LS": "LESOTHO",
    "LR": "LIBERIA",
    "LY": "LIBYA",
    "LI": "LIECHTENSTEIN",
    "LT": "LITHUANIA",
    "LU": "LUXEMBOURG",
    "MO": "MACAO",
    "MK": "MACEDONIA, THE FORMER YUGOSLAV REPUBLIC OF",
    "MG": "MADAGASCAR",
    "MW": "MALAWI",
    "MY": "MALAYSIA",
    "MV": "MALDIVES",
    "ML": "MALI",
    "MT": "MALTA",
    "MH": "MARSHALL ISLANDS",
    "MQ": "MARTINIQUE",
    "MR": "MAURITANIA",
    "MU": "MAURITIUS",
    "YT": "MAYOTTE",
    "MX": "MEXICO",
    "FM": "MICRONESIA, FEDERATED STATES OF",
    "MD": "MOLDOVA, REPUBLIC OF",
    "MC": "MONACO",
    "MN": "MONGOLIA",
    "ME": "MONTENEGRO",
    "MS": "MONTSERRAT",
    "MA": "MOROCCO",
    "MZ": "MOZAMBIQUE",
    "MM": "MYANMAR",
    "NA": "NAMIBIA",
    "NR": "NAURU",
    "NP": "NEPAL",
    "NL": "NETHERLANDS",
    "NC": "NEW CALEDONIA",
    "NZ": "NEW ZEALAND",
    "NI": "NICARAGUA",
    "NE": "NIGER",
    "NG": "NIGERIA",
    "NU": "NIUE",
    "NF": "NORFOLK ISLAND",
    "MP": "NORTHERN MARIANA ISLANDS",
    "NO": "NORWAY",
    "OM": "OMAN",
    "PK": "PAKISTAN",
    "PW": "PALAU",
    "PS": "PALESTINE, STATE OF",
    "PA": "PANAMA",
    "PG": "PAPUA NEW GUINEA",
    "PY": "PARAGUAY",
    "PE": "PERU",
    "PH": "PHILIPPINES",
    "PN": "PITCAIRN",
    "PL": "POLAND",
    "PT": "PORTUGAL",
    "PR": "PUERTO RICO",
    "QA": "QATAR",
    "RE": "RÉUNION",
    "RO": "ROMANIA",
    "RU": "RUSSIAN FEDERATION",
    "RW": "RWANDA",
    "BL": "SAINT BARTHÉLEMY",
    "SH": "SAINT HELENA, ASCENSION AND TRISTAN DA CUNHA",
    "KN": "SAINT KITTS AND NEVIS",
    "LC": "SAINT LUCIA",
    "MF": "SAINT MARTIN (FRENCH PART)",
    "PM": "SAINT PIERRE AND MIQUELON",
    "VC": "SAINT VINCENT AND THE GRENADINES",
    "WS": "SAMOA",
    "SM": "SAN MARINO",
    "ST": "SAO TOME AND PRINCIPE",
    "SA": "SAUDI ARABIA",
    "SN": "SENEGAL",
    "RS": "SERBIA",
    "SC": "SEYCHELLES",
    "SL": "SIERRA LEONE",
    "SG": "SINGAPORE",
    "SX": "SINT MAARTEN (DUTCH PART)",
    "SK": "SLOVAKIA",
    "SI": "SLOVENIA",
    "SB": "SOLOMON ISLANDS",
    "SO": "SOMALIA",
    "ZA": "SOUTH AFRICA",
    "GS": "SOUTH GEORGIA AND THE SOUTH SANDWICH ISLANDS",
    "SS": "SOUTH SUDAN",
    "ES": "SPAIN",
    "LK": "SRI LANKA",
    "SD": "SUDAN",
    "SR": "SURINAME",
    "SJ": "SVALBARD AND JAN MAYEN",
    "SZ": "SWAZILAND",
    "SE": "SWEDEN",
    "CH": "SWITZERLAND",
    "SY": "SYRIAN ARAB REPUBLIC",
    "TW": "TAIWAN, PROVINCE OF CHINA",
    "TJ": "TAJIKISTAN",
    "TZ": "TANZANIA, UNITED REPUBLIC OF",
    "TH": "THAILAND",
    "TL": "TIMOR-LESTE",
    "TG": "TOGO",
    "TK": "TOKELAU",
    "TO": "TONGA",
    "TT": "TRINIDAD AND TOBAGO",
    "TN": "TUNISIA",
    "TR": "TURKEY",
    "TM": "TURKMENISTAN",
    "TC": "TURKS AND CAICOS ISLANDS",
    "TV": "TUVALU",
    "UG": "UGANDA",
    "UA": "UKRAINE",
    "AE": "UNITED ARAB EMIRATES",
    "GB": "UNITED KINGDOM",
    "US": "UNITED STATES",
    "UM": "UNITED STATES MINOR OUTLYING ISLANDS",
    "UY": "URUGUAY",
    "UZ": "UZBEKISTAN",
    "VU": "VANUATU",
    "VE": "VENEZUELA, BOLIVARIAN REPUBLIC OF",
    "VN": "VIET NAM",
    "VG": "VIRGIN ISLANDS, BRITISH",
    "VI": "VIRGIN ISLANDS, U.S.",
    "WF": "WALLIS AND FUTUNA",
    "EH": "WESTERN SAHARA",
    "YE": "YEMEN",
    "ZM": "ZAMBIA",
    "ZW": "ZIMBABWE",
}



#### <font color = blue> The dictionary CC is used to get Country codes. The commented portion in the funcion country works fine, but giving country names with different format than pycountry. So for the time being i am extracting only country names.

In [ ]:
#This function scan the user location and if a country name is present it returns that name.
def country(text):
    for country in pycountry.countries:
        if country.name in text:
            return(country.name)
    #for key in CC.keys(): 
        #if key in text:
            #return(CC[key])
       
            
tweet_df['Country'] =tweet_df['location'].apply(lambda x: country(x))


In [37]:
tweet_df['Country']

1           None
5          Italy
6           None
10          None
12       Ukraine
          ...   
97334       None
97335       None
97336       None
97337       None
97338       None
Name: Country, Length: 54156, dtype: object

#### <font color = blue> The below function with geolocator api is great to find country names from city names. But it only works with fewer data. So for the time being i am only using the above function to het country names from location.

In [36]:
#country name
#This function can extract 
#def country(text):
#    geolocator = Nominatim(user_agent="30042535@students.southwales.ac.uk")
#    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#    loc = str(geolocator.geocode(text,language='en',timeout = None))
#    out = loc.split(',')[-1]
#    return out
  
#tweet_df['Country'] =tweet_df['location'].apply(lambda x: country(x))
    
#tweet_df['Country']



KeyboardInterrupt: 

In [19]:
final_df = tweet_df.dropna(subset = ['Country'])
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13148 entries, 5 to 97333
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   create_at             13148 non-null  object
 1   text                  13148 non-null  object
 2   location              13148 non-null  object
 3   user_name             13148 non-null  object
 4   place                 68 non-null     object
 5   no_url                13148 non-null  object
 6   Mentions              13148 non-null  object
 7   No_User_Mentions      13148 non-null  int64 
 8   cleaned               13148 non-null  object
 9   Tweet_tokenized       13148 non-null  object
 10  final                 13148 non-null  object
 11  tokens                13148 non-null  object
 12  Ukraine               13148 non-null  int64 
 13  Russia                13148 non-null  int64 
 14  UkraineWar            13148 non-null  int64 
 15  IStandWithUkraine     13148 non-null

In [20]:
# To convert tokens back to string for readability

def tostring(text):
    #print(type(text))
    temp = ",".join([str(i) for i in text])
    tweet = temp.replace(',', ' ')
    return tweet
    
    
       
final_df['tweet_text'] =final_df['final'].apply(lambda x: tostring(x))     
#tweet_df['tweet_text']
        


<ipython-input-20-c861128fa809>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['tweet_text'] =final_df['final'].apply(lambda x: tostring(x))


In [21]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13148 entries, 5 to 97333
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   create_at             13148 non-null  object
 1   text                  13148 non-null  object
 2   location              13148 non-null  object
 3   user_name             13148 non-null  object
 4   place                 68 non-null     object
 5   no_url                13148 non-null  object
 6   Mentions              13148 non-null  object
 7   No_User_Mentions      13148 non-null  int64 
 8   cleaned               13148 non-null  object
 9   Tweet_tokenized       13148 non-null  object
 10  final                 13148 non-null  object
 11  tokens                13148 non-null  object
 12  Ukraine               13148 non-null  int64 
 13  Russia                13148 non-null  int64 
 14  UkraineWar            13148 non-null  int64 
 15  IStandWithUkraine     13148 non-null

In [22]:
#Final data frame with desired fields
final = final_df.drop(['create_at','text','user_name','Mentions','location','place','cleaned','final','Tweet_tokenized','tokens','no_url'],axis=1)
final

,No_User_Mentions,Ukraine,Russia,UkraineWar,IStandWithUkraine,StandWithUkraineFlag,IStandWithPutin,IStandWithRussia,RussianUkraineWar,Country,tweet_text
5,1,0,1,0,0,0,0,0,0,Italy,year old boy sees world raped soldier front mother find
12,2,1,0,0,0,0,0,0,0,Ukraine,another victim putin failed presidency illegal war rest peace
20,1,1,0,0,0,0,0,0,0,Ukraine,volunteers glory heroes
28,1,1,0,0,0,0,0,0,0,Spain,catastrophic explosion claimed russian tank eastern front
34,1,0,1,0,0,0,0,0,0,Australia,year old boy sees world raped soldier front mother find
...,...,...,...,...,...,...,...,...,...,...,...
97307,0,0,0,0,0,0,0,0,0,Georgia,russian war crimes
97318,1,0,1,0,0,0,0,0,0,Germany,embassy attacked biological weapon someone threw open famous surstr
97323,1,0,1,0,0,0,0,0,0,United States,year old boy sees world raped soldier front mother find
97324,1,1,0,0,0,0,0,0,0,United Kingdom,update claims liquidated least russian servicemen incl high ranked officers amp elite spet


In [23]:
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13148 entries, 5 to 97333
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   No_User_Mentions      13148 non-null  int64 
 1   Ukraine               13148 non-null  int64 
 2   Russia                13148 non-null  int64 
 3   UkraineWar            13148 non-null  int64 
 4   IStandWithUkraine     13148 non-null  int64 
 5   StandWithUkraineFlag  13148 non-null  int64 
 6   IStandWithPutin       13148 non-null  int64 
 7   IStandWithRussia      13148 non-null  int64 
 8   RussianUkraineWar     13148 non-null  int64 
 9   Country               13148 non-null  object
 10  tweet_text            13148 non-null  object
dtypes: int64(9), object(2)
memory usage: 1.2+ MB


In [24]:
#Convert object type to string
final['Country'] = final['Country'].astype('str') 
final['tweet_text'] = final['tweet_text'].astype('str') 


In [25]:
#Adding an index column to work with redshift.
final['index_col'] = range(1, len(final) + 1)

In [26]:
final = final[['index_col','tweet_text','Country','No_User_Mentions','Ukraine','Russia','UkraineWar','IStandWithUkraine','StandWithUkraineFlag','IStandWithPutin','IStandWithRussia','RussianUkraineWar']]

In [27]:
#Converting all column names to small letters to work with redshift tables
final = final.rename(str.lower, axis='columns')
final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13148 entries, 5 to 97333
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   index_col             13148 non-null  int32 
 1   tweet_text            13148 non-null  object
 2   country               13148 non-null  object
 3   no_user_mentions      13148 non-null  int64 
 4   ukraine               13148 non-null  int64 
 5   russia                13148 non-null  int64 
 6   ukrainewar            13148 non-null  int64 
 7   istandwithukraine     13148 non-null  int64 
 8   standwithukraineflag  13148 non-null  int64 
 9   istandwithputin       13148 non-null  int64 
 10  istandwithrussia      13148 non-null  int64 
 11  russianukrainewar     13148 non-null  int64 
dtypes: int32(1), int64(9), object(2)
memory usage: 1.3+ MB


In [28]:
#Create json string
result = final.to_json(orient="records")

In [29]:
#json string to json file
with open('my_data.json', 'w') as f:
    f.write(result)

In [30]:
#data frame to csv file
final.to_csv("cleaned_tweets.csv", ",", index=False)